In [100]:
import os
import sys
import glob
import datetime
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import xray
import random
import itertools
import urllib
import json
import importlib as imp

from IPython import display

sys.path.append('/nuwa_cluster/home/jackyu/climatools/')
import climatools.dates as climadates
import climatools.viz as climaviz
import climatools.misc as climamisc
import climatools.muths as climaths
import climatools.units as climaunits
import climatools.geocoords as climageocoords
import climatools.prepdatasets as climaprepdata

#from IPython.lib.deepreload import reload as dreload
imp.reload(climadates)
imp.reload(climaviz)
imp.reload(climamisc)
imp.reload(climaths)
imp.reload(climaunits)
imp.reload(climaprepdata)

matplotlib.rcParams.update({'font.size': 13, 
                            'xtick.direction': 'out',
                            'ytick.direction': 'out'})

# ARM95

2 cases considered here are:

* all RRTMG --- RRTMG used for both longwave and shortwave, output sent to CAM
* rad CLIRAD --- CLIRAD used for both longwave and shortwave, output not sent to CAM

RRTMG and CLIRAD are used in each time step to compute radiation.  Both take in the same input from CAM.  Only RRTMG's output at each time step is sent back to CAM.


In [90]:

iopname = 'arm95_LWrrtmgclirad_SWrrtmgclirad'

def case_names():
    'Give the different cases names'
    return ['all RRTMG', 'rad CLIRAD']



def get_cases_difference():
    '''
    define the differences between cases to compute
    '''
    return [('rad CLIRAD', 'all RRTMG'),]



def get_cases():
    '''
    Returns a dictionary where the keys are cases, such
    as a name for a SCAM run, and the values are the datasets (xray)
    for the cases
    '''
    ds = xray.open_dataset('../examples/ARM95/cam5_noavg_LW_rrtmgclirad_SW_rrtmgclirad/camrun.cam.h0.0095-07-18-19800.nc',
                          decode_times = False)
    clirad_names = [k for k in list(ds) if '_clirad' in k]
    rrtmg_names = [name.split('_')[0] for name in clirad_names]
    common_names = [k for k in list(ds) 
                    if k not in (clirad_names + rrtmg_names)]
    
    ds_clirad = xray.Dataset()
    ds_clirad.update(ds[common_names].copy(deep = True), inplace = True)
    ds_clirad.update(ds[clirad_names].copy(deep = True), inplace = True)
    ds_clirad.rename(dict(zip(clirad_names, rrtmg_names)), inplace = True)
    
    ds_rrtmg = xray.Dataset()
    ds_rrtmg.update(ds[common_names].copy(deep = True), inplace = True)
    ds_rrtmg.update(ds[rrtmg_names].copy(deep = True), inplace = True)     
    
       
    return dict(zip(case_names(), [ds_rrtmg, ds_clirad]))




def compute_net_solar_fluxes(datasets):
    '''
    Compute net solar fluxes
    '''
    for casename, ds in datasets.items():
        ds['net_FS'] = ds['FDS'] - ds['FUS']
        ds['net_FS'].attrs['units'] = ds['FDS'].attrs['units']
        ds['net_FS'].attrs['long_name'] = 'net solar flux'
        ds['net_FS'].attrs['case_name'] = casename
        
        ds['net_FSC'] = ds['FDSC'] - ds['FUSC']
        ds['net_FSC'].attrs['units'] = ds['FDSC'].attrs['units']
        ds['net_FSC'].attrs['long_name'] = 'net solar flux (clear-sky)'
        ds['net_FSC'].attrs['case_name'] = casename
    return datasets




def linestyles_by_case():
    '''
    Assign lineplot properties to the cases
    '''
    return dict(zip(case_names(), 
                    [{'colour': 'g', 'linestyle': '--'},
                     {'colour': 'b', 'linestyle': '--'}]))



def get_d3interests():
    '''
    List here all variables of interest that have dimensions (time, lev, lon, lat)
    '''
    d3interests = ['CLOUD', 'QRL', 'QRS', 'Q',
                   'CLDICE', 'CLDLIQ', 
                   'AQSNOW', 'AREI', 'AREL',
                   'AWNC', 'AWNI', 
                   'DCQ', 'DTCOND', 
                   'ICIMR', 'ICWMR', 
                   'NUMLIQ', 'NUMICE', 'T', 'RELHUM',
                   'FUL', 'FDL', 'FULC', 'FDLC', 'net_FS', 'net_FSC',
                   'FUS', 'FDS', 'FUSC', 'FDSC',
                   'TOT_CLD_VISTAU']
    return sorted(d3interests)




def get_d2interests():
    '''
    List here all variables of interest that have dimensions (time, lon, lat)
    '''
    interests = ['FSNT', 'FSNS', 'FLNT', 'FLNS',
                 'CLDLOW', 'CLDMED', 'CLDHGH', 'CLDTOT',
                 'TS', 'TREFHT', 'TROP_T', 'TROP_Z', 'TMQ', 'TGCLDIWP', 'TGCLDLWP', 
                 'PRECT', 'PRECL', 'PRECC',
                 'LHFLX', 'SHFLX',
                'SOLIN']
    return interests






def convert_units(datasets):
    '''
    Convert units for certain fields for all cases in DATASETS.
    INPUT:
    ilev --- level(interface) pressure in mbar
    lev  --- layer(level) pressure in mbar
    datetimes --- time in pandas DatetimeIndex objects
    '''
    for name, ds in datasets.items():
        
        for k in ['TOT_CLD_VISTAU',]:
            if k in ds:
                pass #ds[k].values[ds[k].isnull().values] = - 0.
        
        for k in ['FUS', 'FDS', 'net_FS', 'net_FSC']:
            if k in ds:
                ds[k] *= 1e3
    
        for k in ['QRL', 'QRS', 'DTCOND']:
            if k in ds:
                ds[k] *= 86400
                ds[k].attrs['units'] = 'K/day'
        
        for k in ['Q', ]:
            if k in ds:
                ds[k] *= 1e3
                ds[k].attrs['units'] = 'g/kg'
        
        for k in ['CLDICE', 'CLDLIQ', 'AQSNOW']:
            if k in ds:
                ds[k] *= 1e6
                ds[k].attrs['units'] = 'mg/kg'
                
        for k in ['PRECC', 'PRECL', 'PRECT']:
            if k in ds:
                ds[k] *= 1e3 * 86400
                ds[k].attrs['units'] = 'mm/day'
                
    return datasets

In [109]:
def prep_data(cases):
    '''
    Preprocess data for plotting. 
    pressure and datetimes need to be provided for unit conversion.
    INPUT:
    ilev --- level(interface) pressure [mbar]
    lev --- layer(level) pressure [mbar]
    datetimes --- time in pandas DatetimeIndex objects    
    '''
    interests = get_d3interests() + get_d2interests()
    cases = climaprepdata.convert_pressure_time_units(cases)
    cases = climaprepdata.gather_interests_from_cases(cases, interests)
    cases = convert_units(cases)
    diff_cases = climaprepdata.take_difference_between_cases(cases, get_cases_difference())
    cases, diff_cases = climaprepdata.passon_attrs_casename(cases, diff_cases, interests = interests)
    return cases, diff_cases


In [102]:
################## Variables of dimensions (time, lev, lon, lat) ########################
################################################################################

In [103]:
def interests_using_bwr_cmap():
    return ['QRL', 'QRS', 'DTCOND', 'DCQ']


# for writing plotted figures to disk
def writefig_contourfs(d3sets, diff_d3sets, 
                       interests, 
                       dir_label = 'example_dir_label',
                       file_label = 'example_file_label',
                       fmt = 'png'):
    '''
    For each interest, produce one figure containing contour
    plots for all cases, another figure containing contour 
    Hackish colormap selection here, basically look up a dictionary.
    '''

    for interest in interests:
        
        dirname = 'figs_{}'.format(dir_label)
        if dirname not in glob.glob(dirname):
            os.mkdir(dirname)

        if interest in interests_using_bwr_cmap():
            cmap_name = 'bwr'
        else:
            cmap_name = 'PuBuGn'
            
        fig = climaviz.contourf_interest_for_all_cases(d3sets, interest,
                                           cmap = plt.get_cmap(cmap_name))
        fig.savefig(dirname + '/contourf_{}.{}'.format('_'.join([interest, file_label]), fmt))
        plt.close()
        
        fig = climaviz.contourf_interest_for_all_cases(diff_d3sets, interest,
                                           cmap = plt.get_cmap('bwr'))
        fig.savefig(dirname + '/contourf_{}_diff.{}'.format('_'.join([interest, file_label]), fmt))
        plt.close()
        

        
def writefig_timeaveraged_VSplots(d3sets, diff_d3sets,
                                 interests, 
                                 dir_label = 'example_dir_label', file_label = 'example_file_label',
                                 fmt = 'png'):
    for interest in interests:        
        dirname = 'figs_{}'.format(dir_label)
        if dirname not in glob.glob(dirname):
            os.mkdir(dirname)
            
        fig = climaviz.plotVS_timeaveraged_interest_for_all_cases(d3sets, diff_d3sets, 
                               interest = interest, xscale = 'linear',
                               bot_xlabels_rotate = 10., 
                               top_xlabels_rotate = 20.,
                               linestyles = linestyles_by_case())
        fig.savefig(dirname + '/plot_{}_timeaverage.{}'.format('_'.join([interest, file_label]), fmt))
        plt.close()

In [104]:
def write_d3variable_html_page(interest = 'CLOUD', 
                               dir_label = 'example_dir_label', 
                               file_label = 'example_file_label'):
    figdir = 'figs_' + dir_label
    with open(os.path.join(figdir, '_'.join([interest, file_label]) + '.html'), 
              mode = 'w', encoding = 'utf-8') as f:
        f.write('''
                <img src="contourf_{suffix}.png" height="450">
                <img src="contourf_{suffix}_diff.png" height="450">
                <br>
                <img src="plot_{suffix}_timeaverage.png" height="500">
        '''.format(figdir = figdir, suffix = '_'.join([interest, file_label])))

        
def write_d3variable_links_html(d3sets, interests = ['T'],
                               dir_label = 'example_dir_label', 
                               file_label = 'example_file_label'):
    hyperlinks = ['<a href="figs_{dir_label}/{interest}_{file_label}.html">{interest}: {longname}</a>'\
                 .format(dir_label = dir_label, file_label = file_label,
                 interest = interest, 
                 longname = d3sets[random.choice(list(d3sets.keys()))][interest].attrs['long_name']) \
                 for interest in interests]
    return '<br>'.join(hyperlinks)
    

In [105]:
############### Variables of dimensions (lon, lat, time) ##########################
###################################################################################

In [106]:
# for writing figures to disk
def writefig_plotVS_timeseries(dsets, diff_dsets, interests,
                              sitename, fmt = 'png'):
    for interest in interests:
        fig = climaviz.plotVS_interest_for_all_cases(dsets, diff_dsets, interest = interest,
                                                    linestyles = linestyles_by_case())
        
        dirname = 'figs_{}'.format(sitename)
        if dirname not in glob.glob(dirname):
            os.mkdir(dirname)
            
        fig.savefig(dirname + '/plotVSts_{}.'.format(interest) + fmt)
        plt.close()

def write_d2variable_html_page(iopname = 'mpace', interest = 'FLNS'):
    figdir = 'figs_{iopname}'.format(iopname = iopname)
    with open(os.path.join(figdir, '{iopname}_{interest}.html'\
                           .format(iopname = iopname,
                                   interest = interest)),
             mode = 'w', encoding = 'utf-8') as f:
        f.write('''
        <img src="plotVSts_{interest}.png" width=700>
        '''.format(interest = interest))
        
def write_d2variables_html(d2sets, iopname = 'mpace', interests = ['FLNS']):
    hyperlinks = ['<a href="figs_{iopname}/{iopname}_{interest}.html">{interest}: {longname}</a>'\
                 .format(iopname = iopname, interest = interest,
                        longname = d2sets[random.choice(list(d2sets.keys()))]\
                         [interest].attrs['long_name'])\
                 for interest in interests]
    return '<br>'.join(hyperlinks)


In [110]:
cases = get_cases()
cases = compute_net_solar_fluxes(cases)
dsets, diff_dsets = prep_data(cases) 

# Variables of dimensions (time, ilev, lon, lat)

In [45]:
# plot (time, lev/ilev, lon, lat) variables, write figures to disk and create HTML files

writefig_contourfs(dsets, diff_dsets, get_d3interests(), 
                   dir_label = iopname, file_label = 'pinkslime',
                  fmt = 'png')

writefig_timeaveraged_VSplots(dsets, diff_dsets, get_d3interests(), 
                              dir_label = iopname, file_label = 'pinkslime',
                              fmt = 'png')
# generate HTML page for each interest
for interest in get_d3interests():
    write_d3variable_html_page(interest = interest, dir_label = iopname, file_label = 'pinkslime')


In [47]:
display.HTML(write_d3variable_links_html(dsets, interests = get_d3interests(), dir_label = iopname, file_label = 'pinkslime'))

# Variables of dimensions (lon, lat, time)

In [62]:
# plot (time, lon, lat) variables, and write figures to disk and create HTML files

writefig_plotVS_timeseries(dsets, diff_dsets, get_d2interests(),
                           iopname, fmt = 'png')  

# generate HTML page for each interest
for interest in get_d2interests():
    write_d2variable_html_page(iopname = iopname, interest = interest)


In [66]:
display.HTML(write_d2variables_html(dsets, iopname = iopname, interests = get_d2interests()))

In [68]:
def slice_in_time(cases, time = slice(None)):
    return {casename: ds.sel(time = time) for casename, ds in cases.items()}

In [282]:
which_timeslice = slice('1995-07-19 05:00:00', '1995-07-25 05:00:00')

d3sets_subtime = slice_in_time(d3sets, time = which_timeslice)
diff_d3sets_subtime = slice_in_time(diff_d3sets, time = which_timeslice)

In [283]:
# plot and write figures to disk
#interests = get_d3interests()

#writefig_contourfs(d3sets_subtime, diff_d3sets_subtime, interests, 
#                   dir_label = iopname, file_label = which_timeslice,
#                  fmt = 'png')

#writefig_timeaveraged_VSplots(d3sets_subtime, diff_d3sets_subtime, interests, 
#                              dir_label = iopname, file_label = which_timeslice,
#                              fmt = 'png')


In [284]:
# generate HTML page for each interest
#for interest in get_d3interests():
#    write_d3variable_html_page(interest = interest, dir_label = iopname, file_label = which_timeslice)


#display.HTML((write_d3variable_links_html(interests = interests, dir_label = iopname, file_label = which_timeslice)))

In [285]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')